In [1]:
#Develop Character Language Model
#https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/

In [2]:
#Load Text

In [3]:
#with open(path, 'r', encoding="utf-8") as f:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding="utf-8")
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [4]:
# load text
raw_text = load_doc('ryhme.txt')
print(raw_text)

Sing a song of sixpence,
A pocket full of rye.
Four and twenty blackbirds,
Baked in a pie.
 
When the pie was opened
The birds began to sing;
Wasn't that a dainty dish,
To set before the king.
 
The king was in his counting house,
Counting out his money;
The queen was in the parlour,
Eating bread and honey.
 
The maid was in the garden,
Hanging out the clothes,
When down came a blackbird
And pecked off her nose.


In [5]:
# load text
raw_text = load_doc('news.txt')
print(len(raw_text))

41915785


In [45]:
#print(raw_text[5:500])

In [9]:
"""Clean Text"""

'Clean Text'

In [10]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)

In [11]:
"""Create Sequences"""

'Create Sequences'

In [12]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
	# select sequence of tokens
	seq = raw_text[i-length:i+1]
	# store
	sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 41915774


In [13]:
"""Save Sequences"""

'Save Sequences'

In [14]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w',encoding="utf-8")
	file.write(data)
	file.close()

In [15]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

In [16]:
"""Load the Saved Sequences"""

'Load the Saved Sequences'

In [17]:
# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [18]:
print (lines)

['Sing a song', 'ing a song ', 'ng a song o', 'g a song of', ' a song of ', 'a song of s', ' song of si', 'song of six', 'ong of sixp', 'ng of sixpe', 'g of sixpen', ' of sixpenc', 'of sixpence', 'f sixpence,', ' sixpence, ', 'sixpence, A', 'ixpence, A ', 'xpence, A p', 'pence, A po', 'ence, A poc', 'nce, A pock', 'ce, A pocke', 'e, A pocket', ', A pocket ', ' A pocket f', 'A pocket fu', ' pocket ful', 'pocket full', 'ocket full ', 'cket full o', 'ket full of', 'et full of ', 't full of r', ' full of ry', 'full of rye', 'ull of rye.', 'll of rye. ', 'l of rye. F', ' of rye. Fo', 'of rye. Fou', 'f rye. Four', ' rye. Four ', 'rye. Four a', 'ye. Four an', 'e. Four and', '. Four and ', ' Four and t', 'Four and tw', 'our and twe', 'ur and twen', 'r and twent', ' and twenty', 'and twenty ', 'nd twenty b', 'd twenty bl', ' twenty bla', 'twenty blac', 'wenty black', 'enty blackb', 'nty blackbi', 'ty blackbir', 'y blackbird', ' blackbirds', 'blackbirds,', 'lackbirds, ', 'ackbirds, B', 'ckbirds,

In [ ]:
"""Train the LSTM Model"""

In [18]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')
 
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)
 
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)
 
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)
 
# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))


KeyboardInterrupt



In [36]:
"""Now we can have a look at the Generated Model:"""

'Now we can have a look at the Generated Model:'

In [38]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
 
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text
 
# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))
 
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

Sing a song of sixpence, A poc
king was in his counting house
hello worlem Wlh me  a.ed rhee
